In [5]:
import pickle
import os
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from perses.utils.smallmolecules import  render_protein_residue_atom_mapping
from pkg_resources import resource_filename
from simtk import unit
import argparse
from simtk.openmm import app
import numpy as np

In [6]:
class PointMutationExecutor2(PointMutationExecutor):
    def _solvate(self,
               topology,
               positions,
               water_model,
               phase,
               ionic_strength,
               box_dimensions=None):
        """
        Generate a solvated topology, positions, and system for a given input topology and positions.
        For generating the system, the forcefield files provided in the constructor will be used.
        Parameters
        ----------
        topology : app.Topology
            Topology of the system to solvate
        positions : [n, 3] ndarray of Quantity nm
            the positions of the unsolvated system
        forcefield : SystemGenerator.forcefield
            forcefield file of solvent to add
        water_model : str
            solvent model to use for solvation
        phase : str
            if phase == vacuum, then the complex will not be solvated with water; else, it will be solvated with tip3p
        ionic_strength : float * unit.molar
            the total concentration of ions (both positive and negative) to add using Modeller.
            This does not include ions that are added to neutralize the system.
            Note that only monovalent ions are currently supported.
        Returns
        -------
        solvated_topology : app.Topology
            Topology of the system with added waters
        solvated_positions : [n + 3(n_waters), 3] ndarray of Quantity nm
            Solvated positions
        solvated_system : openmm.System
            The parameterized system, containing a barostat if one was specified.
        """
        modeller = app.Modeller(topology, positions)

        # Now we have to add missing atoms
        if phase != 'vacuum':
#             _logger.info(f"solvating at {ionic_strength} using {water_model}")
            if not box_dimensions:
                modeller.addSolvent(self.system_generator.forcefield, model=water_model, padding=1.1 * unit.nanometers, ionicStrength=ionic_strength)
            else:
                modeller.addSolvent(self.system_generator.forcefield, model=water_model, boxSize=box_dimensions, ionicStrength=ionic_strength)
        else:
            pass

        solvated_topology = modeller.getTopology()
        if box_dimensions:
            solvated_topology.setUnitCellDimensions(box_dimensions)
        solvated_positions = modeller.getPositions()

        # Canonicalize the solvated positions: turn tuples into np.array
        solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)
        solvated_system = self.system_generator.create_system(solvated_topology)
        
        return solvated_topology, solvated_positions, solvated_system

In [7]:
dirs = list(range(5,10))
aminos = ['VAL', 'THR', 'TYR', 'ASP', 'LYS']
outdir = "/data/chodera/zhangi/perses_benchmark/neq/19/"
ala_path = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/ala_vacuum.pdb"

for dir_num, mutant in zip(dirs, aminos):
    solvent_delivery = PointMutationExecutor2(ala_path,
                            '1',
                            '2',
                            mutant,
                            phase='solvent',
                            flatten_torsions=True,
                            flatten_exceptions=True, 
                             generate_unmodified_hybrid_topology_factory=False,
                             generate_rest_capable_hybrid_topology_factory=False,
                             generate_rxn_field_hybrid_topology_factory=True,
                            conduct_endstate_validation=False
                           )

    path = os.path.join(outdir, str(dir_num) + '/')
    print(f"path: {path}")

    apo_htf = solvent_delivery.get_apo_htf()
    with open(os.path.join(path, f"{dir_num}_solvent.pickle"), "wb") as f:
        pickle.dump(apo_htf, f)

    # Render atom map
    atom_map_filename = f'{path}/atom_map.png'
    render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{12: 8}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {9: 9}, {15: 8}, {14: 8}, {13: 8}, {9: 7}, {11: 8}, {10: 8}, {9: 8}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}, {10: 7}]
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{15: 9}
INFO:proposal_generator:{14: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{15: 8}
INFO:proposal_generator:{14: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 10 new atoms
INFO:geometry:	reduced angle potential = 0.530306641332542.
INFO:geometry:	reduced angle potential = 0.008142751541358307.
INFO:geometry:	reduced angle potential = 2.3304406050383593.
INFO:geometry:	reduced angle potential = 0.12631791276310886.
INFO:geometry:	reduced angle potential = 0.019025721462292745.
INFO:geometry:	reduced angle potential = 0.06597067974104848.
INFO:geometry:	reduced angle potential = 0.0435713788099957.
INFO:geometry:	reduced angle potential = 1.1311305588273977.
INFO:geometry:	reduced angle potential = 0.1489479257916522.
INFO:geometry:	reduced angle potential = 0.016612721163536095.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'Per

...

conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced potential before atom placement: 9.117938551333582


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
added energy components: [('CustomBondForce', 0.0006997769715120732), ('CustomAngleForce', 1.9747572915712979), ('CustomTorsionForce', 7.250865032268358), ('CustomBondForce', 13.481284705840158)]


INFO:geometry:total reduced energy added from growth system: 22.70760680665133
INFO:geometry:final reduced energy 31.82554376184619
INFO:geometry:sum of energies: 31.825545357984915
INFO:geometry:magnitude of difference in the energies: 1.5961387198615284e-06
INFO:geometry:Final logp_proposal: -27080.34607147706
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
I

path: /data/chodera/zhangi/perses_benchmark/neq/19/9/


In [12]:
apo_htf

# Test neq code

In [3]:
from simtk import openmm
from openmmtools.integrators import PeriodicNonequilibriumIntegrator, LangevinIntegrator, NonequilibriumLangevinIntegrator, AlchemicalNonequilibriumLangevinIntegrator
from perses.dispersed.utils import configure_platform
from openmmtools import utils, cache, states

conducting subsequent work with the following platform: CUDA


In [4]:
dirs = [5]
aminos = ['VAL']
outdir = "/data/chodera/zhangi/perses_benchmark/neq/19/"
ala_path = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/ala_vacuum.pdb"

for dir_num, mutant in zip(dirs, aminos):
    solvent_delivery = PointMutationExecutor2(ala_path,
                            '1',
                            '2',
                            mutant,
#                             phase='solvent',
                            phase='vacuum',
                            barostat=None,
                            periodic_forcefield_kwargs=None,
                            nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                            flatten_torsions=True,
                            flatten_exceptions=True, 
                             generate_unmodified_hybrid_topology_factory=False,
                             generate_rest_capable_hybrid_topology_factory=False,
                             generate_rxn_field_hybrid_topology_factory=True,
                            conduct_endstate_validation=False
                           )

    path = os.path.join(outdir, str(dir_num) + '/')
    print(f"path: {path}")

    apo_htf = solvent_delivery.get_apo_htf()
    with open(os.path.join(path, f"{dir_num}_solvent.pickle"), "wb") as f:
        pickle.dump(apo_htf, f)

    # Render atom map
    atom_map_filename = f'{path}/atom_map.png'
    render_protein_residue_atom_mapping(apo_htf._topology_proposal, atom_map_filename)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{12: 8}, {15: 9}, {14: 9}, {13: 9}, {12: 9}, {11: 9}, {10: 9}, {9: 9}, {15: 8}, {14: 8}, {13: 8}, {9: 7}, {11: 8}, {10: 8}, {9: 8}, {15: 7}, {14: 7}, {13: 7}, {12: 7}, {11: 7}, {10: 7}]
INFO:proposal_generator:{12: 8}
INFO:proposal_generator:{15: 9}
INFO:proposal_generator:{14: 9}
INFO:proposal_generator:{13: 9}
INFO:proposal_generator:{12: 9}
INFO:proposal_generator:{11: 9}
INFO:proposal_generator:{10: 9}
INFO:proposal_generator:{9: 9}
INFO:proposal_generator:{15: 8}
INFO:proposal_generator:{14: 8}
INFO:proposal_generator:{13: 8}
INFO:proposal_generator:{9: 7}
INFO:proposal_generator:{11: 8}
INFO:proposal_generator:{10: 8}
INFO:proposal_generator:{9: 8}
INFO:proposal_generator:{

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 10 new atoms
INFO:geometry:	reduced angle potential = 0.8170655895714745.
INFO:geometry:	reduced angle potential = 1.9161506561519912.
INFO:geometry:	reduced angle potential = 0.30960975098963056.
INFO:geometry:	reduced angle potential = 0.023541578647019022.
INFO:geometry:	reduced angle potential = 7.969811319019169e-06.
INFO:geometry:	reduced angle potential = 0.8433847061244153.
INFO:geometry:	reduced angle potential = 0.30798208362218366.
INFO:geometry:	reduced angle potential = 0.08306317824459317.
INFO:geometry:	reduced angle potential = 0.26610887369564284.
INFO:geometry:	reduced angle potential = 0.9535085389614826.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 

...

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: 22.70760680665133
INFO:geometry:final reduced energy 31.825543761846138
INFO:geometry:sum of energies: 31.825545357984915
INFO:geometry:magnitude of difference in the energies: 1.5961387767049473e-06
INFO:geometry:Final logp_proposal: -26865.846462563193
INFO:relative:*** Generating RxnHybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 0
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:scale_templates: [['nonscale_

added energy components: [('CustomBondForce', 0.0006997769715120732), ('CustomAngleForce', 1.9747572915712979), ('CustomTorsionForce', 7.250865032268358), ('CustomBondForce', 13.481284705840158)]


INFO:relative:Handling virtual sites...
INFO:relative:	_handle_virtual_sites: numVirtualSites: 0


path: /data/chodera/zhangi/perses_benchmark/neq/19/5/


In [5]:
dir_num = 5
outdir = "/data/chodera/zhangi/perses_benchmark/neq/19/"
path = os.path.join(outdir, str(dir_num) + '/')
with open(os.path.join(path, f"{dir_num}_solvent.pickle"), "rb") as f:
    apo_htf = pickle.load(f)
    system = apo_htf.hybrid_system

In [6]:
apo_htf.hybrid_system.getForce(5).getEnergyFunction()

'U_sterics; U_sterics = 4 * epsilon * x * (x - 1.0); x = (sigma / r_eff)^6; sigma = (sigma1 + sigma2) / 2; sigma1 = (sigma_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 )) + (sigma_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 )); sigma2 = (sigma_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 )) + (sigma_new2 * ( 1 * environment_region2+lambda_0_sterics_new * alchemical_region_02 )); epsilon = sqrt(epsilon1 * epsilon2); epsilon1 = ((epsilon_old1 * ( 1 * environment_region1+lambda_0_sterics_old * alchemical_region_01 )) + (epsilon_new1 * ( 1 * environment_region1+lambda_0_sterics_new * alchemical_region_01 ))) * ( nonscale_lambda_sterics * nonscale_region1 ); epsilon2 = ((epsilon_old2 * ( 1 * environment_region2+lambda_0_sterics_old * alchemical_region_02 )) + (epsilon_new2 * ( 1 * environment_region2+lambda_0_sterics_new * alchemical_region_02 ))) * ( nonscale_lambda_sterics * nonscale_region2 )

In [7]:
apo_htf.hybrid_system.getForce(4).getEnergyFunction()

'U_electrostatics; U_electrostatics = switch * c_RF * u_RF; switch = step(-t) + step(t) * step(1-t) * (1 + (t^3) * (-10 + t * (15 - (6 * t)))); t = (r_eff - r_switch) / (r_cutoff - r_switch); c_RF = 138.935456 * chargeProd; chargeProd = charge1 * charge2; charge1 = ((charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 )) + (charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ))) * ( nonscale_lambda_electrostatics * nonscale_region1 ); charge2 = ((charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 )) + (charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ))) * ( nonscale_lambda_electrostatics * nonscale_region2 ); u_RF = (1 / r_eff) + ((eps_RF - 1) * (r_eff^2)) / ((r_cutoff^3) * (1 + (2 * eps_RF))); r_eff = sqrt(r^2 + w^2); w = is_unique_old * r_cutoff * w_scale * (1 - lambda_0_electrostatics_old) + is_unique_new * w_scale * (1 - la

In [8]:
apo_htf.hybrid_system.getForce(4).setUseSwitchingFunction(False)

In [9]:
apo_htf.hybrid_system.getForce(5).setUseSwitchingFunction(False)

In [15]:
apo_htf.hybrid_system.getForce(4).setUseLongRangeCorrection(False)

In [16]:
apo_htf.hybrid_system.getForce(5).setUseLongRangeCorrection(False)

In [6]:
apo_htf._atom_classes['unique_old_atoms'][0]

{10, 11, 12, 15}

In [5]:
apo_htf._atom_classes['unique_new_atoms'][0]

{2513, 2514, 2515, 2516, 2517, 2518, 2519, 2520, 2521, 2522}

In [6]:
apo_htf._atom_classes['core_atoms'][0]

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 13, 14, 16, 17, 18, 19, 20, 21}

In [21]:
for i in range(apo_htf._topology_proposal.old_system.getForce(3).getNumParticles()):
    print(i, apo_htf._topology_proposal.old_system.getForce(3).getParticleParameters(i))

0 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.06568879999999999, unit=kilojoule/mole)]
1 [Quantity(value=-0.3662, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.4577296, unit=kilojoule/mole)]
2 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.06568879999999999, unit=kilojoule/mole)]
3 [Quantity(value=0.1123, unit=elementary charge), Quantity(value=0.2649532787749369, unit=nanometer), Quantity(value=0.06568879999999999, unit=kilojoule/mole)]
4 [Quantity(value=0.5972, unit=elementary charge), Quantity(value=0.3399669508423535, unit=nanometer), Quantity(value=0.359824, unit=kilojoule/mole)]
5 [Quantity(value=-0.5679, unit=elementary charge), Quantity(value=0.2959921901149463, unit=nanometer), Quantity(value=0.87864, unit=kilojoule/mole)]
6 [Quantity(value=-0.4157, unit=elementary charge), Quantity(valu

1349 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1350 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1351 [Quantity(value=-0.834, unit=elementary charge), Quantity(value=0.3150752406575124, unit=nanometer), Quantity(value=0.635968, unit=kilojoule/mole)]
1352 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1353 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1354 [Quantity(value=-0.834, unit=elementary charge), Quantity(value=0.3150752406575124, unit=nanometer), Quantity(value=0.635968, unit=kilojoule/mole)]
1355 [Quantity(value=0.417, unit=elementary charge), Quantity(value=1.0, unit=nanometer), Quantity(value=0.0, unit=kilojoule/mole)]
1356 [Quantity(value=0.417, unit=e

In [16]:
for atom in apo_htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE1-H1
1 ACE1-CH3
2 ACE1-H2
3 ACE1-H3
4 ACE1-C
5 ACE1-O
6 ALA2-N
7 ALA2-H
8 ALA2-CA
9 ALA2-HA
10 ALA2-CB
11 ALA2-HB2
12 ALA2-HB3
13 ALA2-C
14 ALA2-O
15 ALA2-HB1
2513 ALA2-CB
2514 ALA2-HB
2515 ALA2-CG1
2516 ALA2-HG11
2517 ALA2-HG12
2518 ALA2-HG13
2519 ALA2-CG2
2520 ALA2-HG21
2521 ALA2-HG22
2522 ALA2-HG23
16 NME3-N
17 NME3-H
18 NME3-C
19 NME3-H1
20 NME3-H2
21 NME3-H3
22 HOH4-O
23 HOH4-H1
24 HOH4-H2
25 HOH5-O
26 HOH5-H1
27 HOH5-H2
28 HOH6-O
29 HOH6-H1
30 HOH6-H2
31 HOH7-O
32 HOH7-H1
33 HOH7-H2
34 HOH8-O
35 HOH8-H1
36 HOH8-H2
37 HOH9-O
38 HOH9-H1
39 HOH9-H2
40 HOH10-O
41 HOH10-H1
42 HOH10-H2
43 HOH11-O
44 HOH11-H1
45 HOH11-H2
46 HOH12-O
47 HOH12-H1
48 HOH12-H2
49 HOH13-O
50 HOH13-H1
51 HOH13-H2
52 HOH14-O
53 HOH14-H1
54 HOH14-H2
55 HOH15-O
56 HOH15-H1
57 HOH15-H2
58 HOH16-O
59 HOH16-H1
60 HOH16-H2
61 HOH17-O
62 HOH17-H1
63 HOH17-H2
64 HOH18-O
65 HOH18-H1
66 HOH18-H2
67 HOH19-O
68 HOH19-H1
69 HOH19-H2
70 HOH20-O
71 HOH20-H1
72 HOH20-H2
73 HOH21-O
74 HOH21-H1
75 HOH21-H2
76 HOH22-O
77 HOH

1924 HOH641-O
1925 HOH641-H1
1926 HOH641-H2
1927 HOH642-O
1928 HOH642-H1
1929 HOH642-H2
1930 HOH643-O
1931 HOH643-H1
1932 HOH643-H2
1933 HOH644-O
1934 HOH644-H1
1935 HOH644-H2
1936 HOH645-O
1937 HOH645-H1
1938 HOH645-H2
1939 HOH646-O
1940 HOH646-H1
1941 HOH646-H2
1942 HOH647-O
1943 HOH647-H1
1944 HOH647-H2
1945 HOH648-O
1946 HOH648-H1
1947 HOH648-H2
1948 HOH649-O
1949 HOH649-H1
1950 HOH649-H2
1951 HOH650-O
1952 HOH650-H1
1953 HOH650-H2
1954 HOH651-O
1955 HOH651-H1
1956 HOH651-H2
1957 HOH652-O
1958 HOH652-H1
1959 HOH652-H2
1960 HOH653-O
1961 HOH653-H1
1962 HOH653-H2
1963 HOH654-O
1964 HOH654-H1
1965 HOH654-H2
1966 HOH655-O
1967 HOH655-H1
1968 HOH655-H2
1969 HOH656-O
1970 HOH656-H1
1971 HOH656-H2
1972 HOH657-O
1973 HOH657-H1
1974 HOH657-H2
1975 HOH658-O
1976 HOH658-H1
1977 HOH658-H2
1978 HOH659-O
1979 HOH659-H1
1980 HOH659-H2
1981 HOH660-O
1982 HOH660-H1
1983 HOH660-H2
1984 HOH661-O
1985 HOH661-H1
1986 HOH661-H2
1987 HOH662-O
1988 HOH662-H1
1989 HOH662-H2
1990 HOH663-O
1991 HOH663-H1
199

In [18]:
for i in range(system.getForce(5).getNumParticles()):
    print(i, system.getForce(5).getParticleParameters(i))

0 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2649532787749369, 0.2649532787749369, 0.06568879999999999, 0.06568879999999999)
1 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.3399669508423535, 0.3399669508423535, 0.4577296, 0.4577296)
2 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2649532787749369, 0.2649532787749369, 0.06568879999999999, 0.06568879999999999)
3 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2649532787749369, 0.2649532787749369, 0.06568879999999999, 0.06568879999999999)
4 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.3399669508423535, 0.3399669508423535, 0.359824, 0.359824)
5 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2959921901149463, 0.2959921901149463, 0.87864, 0.87864)
6 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.3249998523775958, 0.3249998523775958, 0.7112800000000001, 0.7112800000000001)
7 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.10690784617684071, 0.10690784617684071, 0.06568879999999999, 0.06568879999999999)
8 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.3399669508423535, 0.3399669508423535, 0.4577296, 0.4577296)
9 (1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.2471353044121301

1405 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0, 0.635968, 0.0)
1406 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1407 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1408 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0, 0.635968, 0.0)
1409 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1410 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1411 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0, 0.635968, 0.0)
1412 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1413 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1414 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0, 0.635968, 0.0)
1415 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1416 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1417 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0, 0.635968, 0.0)
1418 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1419 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0)
1420 (1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.3150752406575124, 0.0,

In [34]:
def compute_w(unique_old1, unique_new1, unique_old2, unique_new2, 
              environment_region, alchemical_region, 
              lambda_old=1, lambda_new=0, 
              r_cutoff=1.4, w_scale=1.):
    # step()
    if (unique_old1 + unique_new1 + unique_old2 + unique_new2 - 0.1) > 0:
        step = 1
    else:
        step = 0
    
    # 1 - old_bool_string1 - new_bool_string1
    x = 1 - (1*environment_region + lambda_old*alchemical_region) - (1*environment_region + lambda_new*alchemical_region)
    
    return step * r_cutoff * x * w_scale

In [16]:
system.getNumParticles()

2523

In [18]:
import itertools

In [24]:
# List of exclusions
unique_news = apo_htf._alchemical_regions_by_type['unique_new_atoms']
unique_olds = apo_htf._alchemical_regions_by_type['unique_old_atoms']

exclusions = []
for new in unique_news:
    for old in unique_olds:
        exclusions.append((old, new))

In [26]:
len(exclusions)

40

In [27]:
# List of nb interactions
interactions = list(itertools.combinations(range(system.getNumParticles()), 2))

In [28]:
len(interactions)

3181503

In [32]:
interactions_filtered = set(interactions) - set(exclusions)

In [39]:
for pair in interactions_filtered:
    params_1 = system.getForce(5).getParticleParameters(pair[0])
    params_2 = system.getForce(5).getParticleParameters(pair[1])
    w = compute_w(params_1[3], params_1[4], params_2[3], params_2[4], params_1[1], params_1[2])
    if w != 0 and w != -1.4:
        print(pair, w)
        
    # Flip particle order
    w = compute_w(params_2[3], params_2[4], params_1[3], params_1[4], params_2[1], params_2[2])
    if w != 0 and w != -1.4:
        print("flipped: ", pair, w)

In [10]:
elec_force = system.getForce(4)
for i in range(elec_force.getNumParticles()):
#     if i in list(apo_htf._atom_classes['unique_old_atoms'][0]) + list(apo_htf._atom_classes['unique_new_atoms'][0]) + list(apo_htf._atom_classes['core_atoms'][0]) : 
#     if i in list(apo_htf._atom_classes['unique_old_atoms'][0]): 
    if i in list(apo_htf._atom_classes['environment_atoms']): 
        params = elec_force.getParticleParameters(i)
        params = list(params)
        params[-1] *= 0
        params[-2] *= 0
        elec_force.setParticleParameters(i, params)



In [144]:
# elec_exceptions_force = system.getForce(6)
# for i in range(elec_exceptions_force.getNumBonds()):
# #     print(elec_exceptions_force.getBondParameters(i))
#     p1, p2, params = elec_exceptions_force.getBondParameters(i)
#     params = list(params)
#     params[-1] *= 0
#     params[-2] *= 0
#     elec_exceptions_force.setBondParameters(i, p1, p2, params)

In [11]:
sterics_force = system.getForce(5)
for i in range(sterics_force.getNumParticles()):
#     if i in list(apo_htf._atom_classes['unique_old_atoms'][0]) + list(apo_htf._atom_classes['unique_new_atoms'][0]) + list(apo_htf._atom_classes['core_atoms'][0]): 
#     if i in list(apo_htf._atom_classes['unique_old_atoms'][0]) : 
    if i in list(apo_htf._atom_classes['environment_atoms']) : 
        params = sterics_force.getParticleParameters(i)
        params = list(params)
        params[-1] *= 0
        params[-2] *= 0
        sterics_force.setParticleParameters(i, params)



In [146]:
# sterics_exceptions_force = system.getForce(7)
# for i in range(sterics_exceptions_force.getNumBonds()):
#     p1, p2, params = sterics_exceptions_force.getBondParameters(i)
#     params = list(params)
#     params[-1] *= 0
#     params[-2] *= 0
#     sterics_exceptions_force.setBondParameters(i, p1, p2, params)

In [109]:

positions = apo_htf.hybrid_positions
box_vectors = apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()

nsteps_eq = 25000
# nsteps_neq = int(1*250000) # 1 ns
nsteps_neq = int(1*25000) # 100 ps
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300.0 * unit.kelvin

x = 'lambda'
ALCHEMICAL_FUNCTIONS = {'lambda_0_bonds_old': f'1 - {x}',
                         'lambda_0_bonds_new': x,
                         'lambda_0_angles_old': f'1 - {x}',
                         'lambda_0_angles_new': x,
                         'lambda_0_torsions_old': f'1 - {x}',
                         'lambda_0_torsions_new': x,
                         'lambda_0_electrostatics_old': f'1 - {x}',
                         'lambda_0_electrostatics_new': x,
                         'lambda_0_electrostatics_exceptions_old': f'1 - {x}', 
                         'lambda_0_electrostatics_exceptions_new': x,
                         'lambda_0_sterics_old': f'1 - {x}',
                         'lambda_0_sterics_new': x,
                         'lambda_0_sterics_exceptions_old': f'1 - {x}',
                         'lambda_0_sterics_exceptions_new': x}

# Set up integrator
integrator = PeriodicNonequilibriumIntegrator(ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)
# integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
# integrator = LangevinIntegrator(temperature=temperature)
# integrator = NonequilibriumLangevinIntegrator(temperature=temperature)
# integrator = AlchemicalNonequilibriumLangevinIntegrator(ALCHEMICAL_FUNCTIONS, temperature=temperature)

# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*box_vectors)
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)


In [17]:
for param, val in context.getParameters().items():
    if 'old' in param:
        context.setParameter(param, 0.0)
    elif 'new' in param:
        context.setParameter(param, 1.0)

In [18]:
for param, val in context.getParameters().items():
    print(param, val)

MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_0_angles_new 1.0
lambda_0_angles_old 0.0
lambda_0_bonds_new 1.0
lambda_0_bonds_old 0.0
lambda_0_electrostatics_exceptions_new 1.0
lambda_0_electrostatics_exceptions_old 0.0
lambda_0_electrostatics_new 1.0
lambda_0_electrostatics_old 0.0
lambda_0_sterics_exceptions_new 1.0
lambda_0_sterics_exceptions_old 0.0
lambda_0_sterics_new 1.0
lambda_0_sterics_old 0.0
lambda_0_torsions_new 1.0
lambda_0_torsions_old 0.0
nonscale_lambda_angles 1.0
nonscale_lambda_bonds 1.0
nonscale_lambda_electrostatics 1.0
nonscale_lambda_electrostatics_exceptions 1.0
nonscale_lambda_sterics 1.0
nonscale_lambda_sterics_exceptions 1.0
nonscale_lambda_torsions 1.0


In [110]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



In [111]:
# Run eq forward (0 -> 1)
positions = []
energies = []
for step in range(int(nsteps_eq/10)):
    print(step)
    integrator.step(10)
#     energy = context.getState(getEnergy=True, enforcePeriodicBox=False).getPotentialEnergy()
#     energies.append(energy)
    
# #     if step > 2000:
#     pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#     old_pos = np.asarray(apo_htf.old_positions(pos))
# #     old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
# #     old_pos_solute = old_traj.atom_slice(old_traj.top.select("not water")).xyz[0]

#     positions.append(old_pos)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070


In [12]:
import mdtraj as md

In [13]:
traj = md.Trajectory(positions,  md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()])
vectors = np.array([box_vectors]) 
traj.unitcell_vectors = np.repeat(vectors, 803, axis=0)

traj = traj.image_molecules()
traj.save("traj.dcd")
traj[0].save("traj.pdb")

/home/zhangi/miniconda3/envs/perses-rxn-field/lib/python3.8/site-packages/numpy/core/_methods.py:178: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [112]:
# Run neq forward (0 -> 1)
forward_works_master = list()
forward_neq_old, forward_neq_new = list(), list()
forward_works = [integrator.get_protocol_work(dimensionless=True)]
positions_old = []
positions_new = []
for fwd_step in range(int(nsteps_neq / 2500)):
    integrator.step(2500)
    print(f"Forward neq: {fwd_step*2500} completed")
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    old_pos = np.asarray(apo_htf.old_positions(pos))
    new_pos = np.asarray(apo_htf.new_positions(pos))

    positions_old.append(old_pos)
    positions_new.append(new_pos)
    
    print(context.getParameter('lambda_0_electrostatics_old'))
    print(context.getParameter('lambda_0_electrostatics_new'))



#     forward_works.append(integrator.get_protocol_work(dimensionless=True))
    
#     pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
#     old_pos = np.asarray(htf.old_positions(pos))
#     old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(htf._topology_proposal.old_topology))
#     old_pos_solute = old_traj.atom_slice(old_traj.top.select("not water")).xyz[0]
    
#     new_pos = np.asarray(htf.new_positions(pos))
#     new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(htf._topology_proposal.new_topology))
#     new_pos_solute = new_traj.atom_slice(new_traj.top.select("not water")).xyz[0]
    
#     forward_neq_old.append(old_pos_solute)
#     forward_neq_new.append(new_pos_solute)
# forward_works_master.append(forward_works)

Forward neq: 0 completed
0.9000000000000034
0.0999999999999966
Forward neq: 2500 completed
0.7999999999999814
0.20000000000001855
Forward neq: 5000 completed
0.6999999999999855
0.3000000000000145
Forward neq: 7500 completed
0.6000000000000243
0.3999999999999757
Forward neq: 10000 completed
0.5000000000000631
0.49999999999993694
Forward neq: 12500 completed
0.39999999999996305
0.600000000000037
Forward neq: 15000 completed
0.29999999999986304
0.700000000000137
Forward neq: 17500 completed
0.19999999999976303
0.800000000000237
Forward neq: 20000 completed
0.09999999999966303
0.900000000000337
Forward neq: 22500 completed
-4.369837824924616e-13
1.000000000000437


In [90]:
for pos in positions[9]:
    print(pos)

[0.26869541 1.21144319 1.77831627]
[0.25724484 1.30579308 1.72494977]
[0.15244028 1.31650627 1.75291819]
[0.26843593 1.29796694 1.6168086 ]
[0.34429515 1.41456133 1.7748412 ]
[0.38605231 1.50772987 1.6984281 ]
[0.36932213 1.40389062 1.9077305 ]
[0.33060633 1.32785263 1.96177038]
[0.47267563 1.49715096 1.96293126]
[0.43969476 1.59889427 1.9419179 ]
[0.4817953  1.47410451 2.11686205]
[0.6115661  1.49525852 1.89359761]
[0.66332433 1.39045933 1.84856998]
[0.52626579 1.37474057 2.12235594]
[0.56438196 1.5861606  2.17458568]
[0.6593444  1.60286247 2.12375083]
[0.50602388 1.67804608 2.16889408]
[0.57894389 1.55646209 2.27844592]
[0.34903085 1.48168904 2.19442336]
[0.36821338 1.47872343 2.30168116]
[0.28694222 1.56405258 2.15917777]
[0.30101243 1.38814222 2.16571408]
[0.67470415 1.61318799 1.87632963]
[0.63199928 1.69443435 1.91847607]
[0.78587363 1.64379068 1.79035661]
[0.74132901 1.66549759 1.69327115]
[0.84119469 1.7327902  1.82035104]
[0.85536179 1.55995921 1.7853886 ]
[2.80582607 0.499858

[2.71122657 2.19205838 2.28680853]
[2.56064307 2.19555443 2.27159879]
[1.19995459 0.95230994 2.26091944]
[1.12552416 0.95623644 2.20086105]
[1.17040837 0.89550512 2.33207105]
[1.80118901 0.2413435  1.13199114]
[1.84722996 0.32196442 1.10869276]
[1.7294291  0.27013686 1.18841611]
[1.50341839 1.20482006 1.70773279]
[1.47622264 1.19581356 1.79906511]
[1.58767371 1.15962132 1.70321245]
[-0.0383749   2.61923768  0.85792058]
[-0.00280648  2.65121242  0.77500603]
[-0.03815288  2.69587229  0.91527392]
[2.58479417 2.38865652 0.46829144]
[2.62759446 2.30510393 0.44959872]
[2.65691265 2.45060251 0.47942422]
[0.90493309 2.29418819 1.56163397]
[0.97851864 2.29309689 1.500427  ]
[0.85924096 2.37615597 1.54277052]
[0.33476054 1.42419731 0.04586496]
[ 0.33374046  1.38742836 -0.04250541]
[0.27542714 1.36814606 0.09586619]
[0.92051152 1.95746535 1.57073129]
[0.82708208 1.97610264 1.58000001]
[0.96020983 1.99386615 1.64985994]
[1.98252702 0.74650931 0.8565579 ]
[1.90474975 0.71419845 0.90204441]
[1.96255

[2.83437762 1.3075901  2.16536265]
[2.68609605 1.29400381 2.13803241]
[0.25105638 1.25700035 0.39309699]
[0.18869917 1.21494447 0.45230171]
[0.21038262 1.25031305 0.30670693]
[0.63901082 2.1556035  1.63346131]
[0.70290681 2.21031285 1.587783  ]
[0.56815863 2.14357502 1.57023472]
[0.81126657 2.15263577 2.31757123]
[0.7945283  2.16007152 2.41152259]
[0.76829545 2.07103941 2.29192296]
[2.72017412 2.44815489 0.74008966]
[2.76479847 2.5172808  0.78900381]
[2.78535048 2.37851051 0.73208832]
[0.09733319 2.80364915 2.27193808]
[0.04159997 2.80494232 2.1941277 ]
[0.08261415 2.71731635 2.31056858]
[2.61619303 0.27365457 0.25561692]
[2.52531063 0.30369953 0.25570398]
[2.61775052 0.20209263 0.19206552]
[1.17473637 0.98630224 1.23984264]
[1.09181674 0.94341492 1.21869244]
[1.1520829  1.05046183 1.30716786]
[0.77774745 1.10894688 0.01539742]
[ 0.68799698  1.08114702 -0.00288748]
[0.77044514 1.16376805 0.09352318]
[1.17606507 2.61970198 2.80247291]
[1.16235878 2.52544581 2.792974  ]
[1.21120159 2.647

In [91]:
import mdtraj as md

In [113]:
traj = md.Trajectory(positions_new,  md.Topology.from_openmm(apo_htf._topology_proposal.new_topology))
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()])
vectors = np.array([box_vectors]) 
traj.unitcell_vectors = np.repeat(vectors, 10, axis=0)

traj = traj.image_molecules()
traj.save("traj_neq_new.dcd")
traj[0].save("traj_neq_new.pdb")

traj = md.Trajectory(positions_old,  md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()])
vectors = np.array([box_vectors]) 
traj.unitcell_vectors = np.repeat(vectors, 10, axis=0)

traj = traj.image_molecules()
traj.save("traj_neq_old.dcd")
traj[0].save("traj_neq_old.pdb")

In [95]:
for pos in context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True):
    print(pos)

[0.26869541 1.21144319 1.77831627] nm
[0.25724484 1.30579308 1.72494977] nm
[0.15244028 1.31650627 1.75291819] nm
[0.26843593 1.29796694 1.6168086 ] nm
[0.34429515 1.41456133 1.7748412 ] nm
[0.38605231 1.50772987 1.6984281 ] nm
[0.36932213 1.40389062 1.9077305 ] nm
[0.33060633 1.32785263 1.96177038] nm
[0.47267563 1.49715096 1.96293126] nm
[0.43969476 1.59889427 1.9419179 ] nm
[0.50014989 1.48163224 2.11679144] nm
[0.46117882 1.38762784 2.15584819] nm
[0.60640218 1.4939011  2.13778969] nm
[0.6115661  1.49525852 1.89359761] nm
[0.66332433 1.39045933 1.84856998] nm
[0.43954209 1.56294926 2.15673187] nm
[0.67470415 1.61318799 1.87632963] nm
[0.63199928 1.69443435 1.91847607] nm
[0.78587363 1.64379068 1.79035661] nm
[0.74132901 1.66549759 1.69327115] nm
[0.84119469 1.7327902  1.82035104] nm
[0.85536179 1.55995921 1.7853886 ] nm
[2.80582607 0.49985825 0.26615636] nm
[2.86081513 0.50254688 0.34445894] nm
[2.74164947 0.43118197 0.28424644] nm
[2.32361229 0.48209918 1.94436986] nm
[2.41724991 

[0.54995266 2.99624447 2.82683686] nm
[0.50804332 2.8655078  2.76303652] nm
[2.98405343 1.78024782 2.86104531] nm
[2.97565967 1.72590847 2.93939771] nm
[3.03428628 1.7264834  2.79982141] nm
[1.82681865 2.83083659 0.38305104] nm
[1.78001649 2.76484116 0.43420257] nm
[1.86500635 2.78202319 0.31010401] nm
[0.33106209 1.7685821  2.64661953] nm
[0.42322691 1.74413988 2.65501853] nm
[0.28324346 1.68665889 2.65943686] nm
[1.0295345  2.46947795 2.42387498] nm
[1.04422185 2.5574183  2.38904548] nm
[1.05503205 2.41087491 2.35261575] nm
[2.07291241 0.22827966 1.6180269 ] nm
[2.02404483 0.14965618 1.59368298] nm
[2.12785414 0.20063843 1.69137324] nm
[2.89757917 1.26727791 1.4733533 ] nm
[2.93690054 1.24786595 1.38826909] nm
[2.96313025 1.32043287 1.51851883] nm
[2.13119275 0.2458694  2.75406687] nm
[2.15727558 0.16191165 2.79192318] nm
[2.11445512 0.3021247  2.8296811 ] nm
[1.16614639 2.75810397 2.34996843] nm
[1.23136526 2.69434457 2.32092438] nm
[1.18529554 2.83689316 2.29909711] nm
[1.31863689 

[2.593545   1.24789057 1.41234092] nm
[2.71475873 1.29915023 1.48716492] nm
[2.04880223 2.81541541 1.06366252] nm
[2.09497292 2.81408484 1.14750055] nm
[2.08690935 2.74291517 1.01412586] nm
[2.75005715 0.11300561 0.45389783] nm
[2.78026975 0.18753792 0.50580595] nm
[2.71256535 0.15228283 0.37506902] nm
[ 2.6471938   1.39414628 -0.0642511 ] nm
[ 2.58573159  1.32078173 -0.06271557] nm
[2.63002684 1.44154278 0.01711957] nm
[1.72280358 0.83821331 0.58902749] nm
[1.790308   0.81768262 0.52434393] nm
[1.68153667 0.75400147 0.6082037 ] nm
[0.16891382 2.26866958 0.32097765] nm
[0.11852425 2.31936847 0.25731578] nm
[0.14239875 2.1779553  0.30580573] nm
[1.88855073 2.99251265 1.4463835 ] nm
[1.86808066 3.00821017 1.35420496] nm
[1.91621541 2.90093285 1.44956518] nm
[0.09453935 2.52708186 0.58992332] nm
[0.15957319 2.58937702 0.5574852 ] nm
[0.1258797  2.50286424 0.67706462] nm
[1.74418376 1.04881055 1.74309476] nm
[1.77418175 1.09239633 1.82286141] nm
[1.80027512 1.08400054 1.67397354] nm
[1.590

[1.29463969 0.95387194 1.08769691] nm
[2.12672073 0.74970129 1.43492587] nm
[2.20733405 0.75815531 1.4858409 ] nm
[2.08936717 0.83782994 1.43431691] nm
[1.80694981 1.0266395  2.60302887] nm
[1.81712904 1.10045274 2.66311471] nm
[1.72697955 1.04627254 2.55422703] nm
[1.6451948  2.8400173  1.26860407] nm
[1.67901393 2.86845201 1.18369204] nm
[1.68812961 2.75597826 1.28461533] nm
[1.46583081 0.56511121 1.4351127 ] nm
[1.40706325 0.61835049 1.48872493] nm
[1.48803422 0.62126386 1.36084153] nm
[0.72856982 2.29237414 0.61194949] nm
[0.76997244 2.20674142 0.60121732] nm
[0.80122791 2.35210289 0.62971453] nm
[0.80306776 1.76974078 0.57412967] nm
[0.78617763 1.69349777 0.62948349] nm
[0.82747086 1.73283809 0.48924741] nm
[1.42408886 0.85209112 0.02734336] nm
[1.40145833 0.77200834 0.0746395 ] nm
[1.45375045 0.91256184 0.09535661] nm
[2.16353144 1.92213968 1.08867725] nm
[2.10804354 1.86056705 1.04080009] nm
[2.21959138 1.96109913 1.02158208] nm
[0.93278604 1.41935118 0.89463913] nm
[0.92530113 

In [96]:
integrator.step(1)

In [97]:
for pos in context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True):
    print(pos)

[ 865532.52277536 -390568.44707593 1772084.42895246] nm
[-5016366.02283724 -1518299.08039916 -2553203.79710391] nm
[2255167.76759541 1681771.67357903 4060979.01883071] nm
[  684214.22548316  -139568.56408085 -3879306.32713912] nm
[ 2855.55961074 -2853.79824357 -2853.43929359] nm
[ 2143.58170829  2144.70520286 -2141.49848384] nm
[ 2283.67353574  2284.71448329 -2281.39780352] nm
[ 2283.76636746  2284.7423456  -2281.46382893] nm
[0.47075452 1.49709233 1.96579472] nm
[0.44017663 1.59805982 1.93838013] nm
[ 3141413.06799253 -3242951.13148274  -694300.24470576] nm
[ 2146529.5932551   6037438.54264169 -2129062.2904836 ] nm
[ -1731337.38695482 -15324167.023838      897071.9218256 ] nm
[0.60893816 1.49402063 1.8920769 ] nm
[ 2143.85551344 -2141.80731093 -2141.34817657] nm
[-2806524.92410691 11755085.26864594  1767197.96141377] nm
[ 4.56727120e+03  1.69622769e+00 -4.56471726e+03] nm
[ 4.56753045e+03  1.47469752e+00 -4.56497389e+03] nm
[-3877755.27884134  3499852.92704823   615650.39624919] nm
[-

[nan nan nan] nm
[2.06982735 0.2279053  1.6194415 ] nm
[2.02774659 0.14559703 1.59460452] nm
[2.12599267 0.20477611 1.69341998] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[2.13397555 0.24586204 2.75168238] nm
[2.16094634 0.16307871 2.79145442] nm
[2.11681833 0.30371805 2.82598332] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[0.0604812  2.65254945 1.71407662] nm
[0.03005076 2.6698502  1.80316641] nm
[0.04179824 2.73331789 1.66622554] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[1.7282201  2.54221941 1.35452942] nm
[1.72095373 2.46877865 1.29357064] nm
[1.82225744 2.55153978 1.36977501] nm
[2.80953474 2.01342063 0.80328093] nm
[2.80700217 2.07252749 0.87852907] nm
[2.90279949 2.00035341 0.78615632] nm
[1.90194451 1.64130634 1.44584883] nm
[1.92296655 1.5488

[1.93716571 2.78881752 2.31910095] nm
[1.98396831 2.84555344 2.38036163] nm
[2.00017648 2.72024466 2.29697125] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[0.21718383 0.39076814 2.28554844] nm
[0.18266196 0.3018957  2.2940484 ] nm
[0.2878302  0.38304105 2.2214259 ] nm
[2.04528302 1.03259429 2.43718983] nm
[1.9805348  1.02802182 2.50753952] nm
[2.00446677 0.98639069 2.36396702] nm
[1.33252863 0.06171397 0.16108896] nm
[ 1.33333911 -0.00659749  0.22813526] nm
[1.28233576 0.13318972 0.20025797] nm
[1.52091736 2.72283247 0.83541981] nm
[1.47216689 2.76774824 0.90447258] nm
[1.58804356 2.67295394 0.88198723] nm
[1.81323607 1.00281871 0.80515629] nm
[1.77831553 0.95720644 0.72859001] nm
[1.73663088 1.01980352 0.85997854] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[0.42402621 0.7532429  0.83126275] nm
[0.32996903 0.75015572 0.81376887] nm
[0.43078544 0.7884299  0.9200237 ] nm
[1.93406976 0.07185443 0.73065514] nm
[1.84074271 0.09004589 0.74167561] nm
[1.93872045 0.0167816

[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[2.13212199 2.40593052 0.72576391] nm
[2.16613583 2.48151407 0.67788382] nm
[2.03702686 2.41313414 0.71755763] nm
[1.23307334 1.17973834 0.69749804] nm
[1.26578815 1.18276702 0.78740292] nm
[1.16089877 1.24259671 0.69609259] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[0.92200363 0.86690042 0.68209001] nm
[0.90633529 0.7731727  0.67060368] nm
[0.95725746 0.89582949 0.59793186] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[2.4125764  0.3698247  1.66160967] nm
[2.33587374 0.31525975 1.67897876] nm
[2.38256217 0.43345513 1.59670487] nm
[2.04273311 2.94671789 0.47460308] nm
[1.97191389 2.88922523 0.44559358] nm
[2.08020307 2.98137677 0.39362726] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[nan nan nan] nm
[1.36984746 2.8590483  0.65083622] nm
[1.44551918 2.8119

In [83]:
# Run eq reverse (1 -> 0)
positions_eq = []
for step in range(nsteps_eq):
    print(step)
    integrator.step(1)
    
    pos = context.getState(getPositions=True, enforcePeriodicBox=False).getPositions(asNumpy=True)
    old_pos = np.asarray(apo_htf.old_positions(pos))
#     old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
#     old_pos_solute = old_traj.atom_slice(old_traj.top.select("not water")).xyz[0]

    positions_eq.append(old_pos)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


OpenMMException: Particle coordinate is nan

In [84]:
for pos in positions_eq[0]:
    print(pos)

[ -1107350.46942971  -9178214.32481527 -12844626.34680727]
[ 7333556.43155102  6813591.01099538 19481265.02203995]
[-3808570.61810029 -4826039.92337646 -7804731.16131099]
[-663726.42814113 8853290.82857317 5872827.19775283]
[-2854.93358562  2855.28682676  2857.61884122]
[ 2143.59584909  2143.23470849 -2140.82769061]
[ 2283.44075627 -2283.09726727  2285.75550169]
[ 2283.86608644 -2283.24893487  2285.74045615]
[0.31797866 0.31013703 2.35888634]
[0.34727995 0.28552785 2.2568235 ]
[ 1071027.59911333  1892002.93046853 -1257717.00856137]
[   21809.97711364 -1202054.71125     1497661.86071626]
[1102612.61480798 1139857.66572301 -773297.69737799]
[0.44056101 0.3259339  2.4395964 ]
[-2142.77210082 -2142.88478692 -2140.63588597]
[-1953941.8419697  -1355739.88830979   246768.63965226]
[ 4.76783370e-01  3.37488282e-01 -4.56449735e+03]
[ 7.11412096e-01  4.39799998e-01 -4.56388140e+03]
[13961872.43385234  -748502.37433716 -2443610.05806736]
[-4741926.97919264 -5222412.26606904  6474210.15384901]
[ -

[1.97013398 1.7734117  0.70890075]
[2.04694564 1.84200625 0.5979336 ]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[2.76729205 1.00973206 0.88914637]
[2.69292702 1.03794028 0.83588797]
[2.72785948 0.96131847 0.96169645]
[1.88768557 1.39440264 1.77795452]
[1.82201012 1.44856241 1.82172406]
[1.83710122 1.33765652 1.71978735]
[0.56408791 2.10665705 1.66970299]
[0.61085715 2.1680317  1.72634316]
[0.50046557 2.06443886 1.72742768]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[2.87049573 1.8855268  0.43228101]
[2.90604013 1.84326497 0.35409629]
[2.79551529 1.93577373 0.40041411]
[1.15517143 2.14084544 0.48731065]
[1.10951674 2.08171194 0.42746755]
[1.11163819 2.12801481 0.57158728]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[2.8580759  1.95717    0.88594609]
[2.76565397 1.98207322 0.88652381]
[2.89514617 2.00423526 0.81129373]
[2.15736142 1.69017127 0.05526787]
[2.10167189 1.76646966 0.03979045]
[2.11285351 1.61842838 0.01016441]
[1.61142143 2.97232584 1.13

[1.06114599 1.24726878 1.5404651 ]
[1.12345301 1.21787319 1.40565899]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[2.2710596  2.29498087 1.1254917 ]
[2.32542156 2.37311768 1.11540535]
[2.30656399 2.25084033 1.20264978]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[0.93894376 1.76017657 1.81218043]
[0.88985057 1.84142495 1.79989718]
[0.96122938 1.73195786 1.72347094]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.52214771 2.43746714 2.8191693 ]
[1.57933892 2.49951496 2.77398577]
[1.47235734 2.49120243 2.880779  ]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.73255404 2.25044128 1.20806269]
[1.65290223 2.21179889 1.17166663]
[1.78261489 2.17576062 1.24091126]
[ 2.03101696  1.94707565 -0.00225838]
[2.06796061 2.02484377 0.0395699 ]
[ 1.94726843  1.97655901 -0.03802475]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[2.39033964 1.8170612

[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.49678269 2.75730611 1.90313641]
[1.43789616 2.70433604 1.95688458]
[1.47581428 2.84772332 1.92653239]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.84009836 2.35168425 2.71970374]
[1.92597647 2.35823752 2.67793865]
[1.80132123 2.43848965 2.7085918 ]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.46635754 2.3442593  0.35655461]
[1.4703669  2.25131017 0.37906464]
[1.37546395 2.36912296 0.37336151]
[0.87051572 2.9343554  1.0134997 ]
[0.82841849 2.85783181 1.0526696 ]
[0.95175357 2.90044427 0.97591293]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[1.34205193 1.94242045 0.93204045]
[1.41527183 2.00137351 0.91399324]
[1.36814075 1.85906438 0.89288107]
[1.44273718 2.26061584 1.11959045]
[1.39833628 2.17633337 1.1102447 ]
[1.4352326  2.30139146 1.03331561]
[1.63075085 2.92473054 2.1603363 ]

In [12]:
import mdtraj as md

In [69]:
traj = md.Trajectory(positions_eq[:11],  md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
box_vectors = np.array([val.value_in_unit_system(unit.md_unit_system) for val in apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()])
vectors = np.array([box_vectors]) 
traj.unitcell_vectors = np.repeat(vectors, 11, axis=0)

traj = traj.image_molecules()
traj.save("traj.dcd")
traj[0].save("traj.pdb")

ValueError: Particle position is NaN

In [36]:
for param, val in context.getParameters().items():
    print(param, val)

MonteCarloPressure 1.01325
MonteCarloTemperature 300.0
lambda_0_angles_new 1.000000000000437
lambda_0_angles_old -4.369837824924616e-13
lambda_0_bonds_new 1.000000000000437
lambda_0_bonds_old -4.369837824924616e-13
lambda_0_electrostatics_exceptions_new 1.000000000000437
lambda_0_electrostatics_exceptions_old -4.369837824924616e-13
lambda_0_electrostatics_new 1.000000000000437
lambda_0_electrostatics_old -4.369837824924616e-13
lambda_0_sterics_exceptions_new 1.000000000000437
lambda_0_sterics_exceptions_old -4.369837824924616e-13
lambda_0_sterics_new 1.000000000000437
lambda_0_sterics_old -4.369837824924616e-13
lambda_0_torsions_new 1.000000000000437
lambda_0_torsions_old -4.369837824924616e-13
nonscale_lambda_angles 1.0
nonscale_lambda_bonds 1.0
nonscale_lambda_electrostatics 1.0
nonscale_lambda_electrostatics_exceptions 1.0
nonscale_lambda_sterics 1.0
nonscale_lambda_sterics_exceptions 1.0
nonscale_lambda_torsions 1.0


# Check PDBs

In [24]:
import mdtraj as md

In [27]:
old_pos = np.asarray(apo_htf.old_positions(apo_htf.hybrid_positions))
old_traj = md.Trajectory(old_pos, md.Topology.from_openmm(apo_htf._topology_proposal.old_topology))
old_traj[0].save("old.pdb")

In [28]:
new_pos = np.asarray(apo_htf.new_positions(apo_htf.hybrid_positions))
new_traj = md.Trajectory(new_pos, md.Topology.from_openmm(apo_htf._topology_proposal.new_topology))
new_traj[0].save("new.pdb")